<a href="https://colab.research.google.com/github/imirghania/online_regression_task/blob/main/ml_remaining_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Expected completion time

Imagine being a team lead of an ML group. Recently a fresh graduate joins your team, as a part of the onboarding you gave him a task to implement module that predicts the expected remaining completion time for series of similar tasks like for example sequence of gradient descent updates applied to the neural network during learning (`tqdm` has ETA for that, but you believe that it is not good enough). After a brief time he returned to you with the suggestion that fitting transformer neural networks for the task is the perfect solution for the problem as transformers and their variants are SOTA for many of sequence processing tasks. Although frustrated with the degree of overkill you decided to demonstrate your soft skills and encourage him to implement his idea while providing simple baseline based on linear regression in online format.

So the problem is as follows: you observe sequence of $n$ events that occurs at timestamps $t_1, t_2, \ldots, t_n$, at each iteration $i$ you want to have an estimate of $t_n-t_i$ based on linear regression constructed from $t_1, t_2, \ldots, t_i$, i.e. prediction $y_j$ is made by the formula
$$
y_j=kj+b
$$
and $k, b$ are chosen in a way that minimizes
$$
\sum_{j=1}^i(t_j-y_j)^2
$$

## Mathematical formulation

$L = \sum_{j=1}^{i}(t_j - y_j)^2 \quad = \quad \sum_{j=1}^{i}(t_j - (kj+b))^2$


$\frac{\partial L}{\partial k} \quad = \quad 2\sum_{j=1}^{i} - j(t_j-(kj+b)) = \quad 2\sum_{j=1}^{i}j((kj+b) - t_j)$

$\frac{\partial L}{\partial k} = 0 \quad \Rightarrow \quad \sum_{j=1}^{i}j((kj+b) - t_j) = 0$

$\Rightarrow \quad \sum_{j=1}^{i}(kj^2 + bj - jt_j) = 0$

$\Rightarrow \quad k\sum_{j=1}^{i}j^2 + b\sum_{j=1}^{i}j - \sum_{j=1}^{i}jt_j = 0$

$\therefore \quad k\sum_{j=1}^{i}j^2 + b\sum_{j=1}^{i}j = \sum_{j=1}^{i}jt_j$

$\frac{\partial L}{\partial b} \quad = \quad 2\sum_{j=1}^{i}t_j - (kj+b) = \quad 2\sum_{j=1}^{i}t_j - kj - b$

$\frac{\partial L}{\partial b} = 0 \quad \Rightarrow \quad \sum_{j=1}^{i}t_j-kj-b = 0$

$\Rightarrow \quad \sum_{j=1}^{i}t_j - \sum_{j=1}^{i}kj - \sum_{j=1}^{i}b = 0$

$\Rightarrow \quad \sum_{j=1}^{i}t_j - k\sum_{j=1}^{i}j - i*b = 0$


$\therefore \quad k\sum_{j=1}^{i}j + i*b = \sum_{j=1}^{i}t_j$


### By solving

$$
\begin{align*}
k\sum_{j=1}^{i}j^2+b\sum_{j=1}^{i}j &= \sum_{j=1}^{i}jt_j \\
k\sum_{j=1}^{i}j + b*i &= \sum_{j=1}^{i}t_j
\end{align*}
$$

$\Large b = \frac{\sum_{j=1}^{i}t_j \quad - \quad k\sum_{j=1}^{i}j}{i}$

$\Large k = \frac{i\sum_{j=1}^{i}jt_j \quad - \quad \sum_{j=1}^{i}j\sum_{j=1}^{i}t_j}{i\sum_{j=1}^{i}j^2 \quad - \quad (\sum_{j=1}^{i}j)^2}$

In [22]:
class ETAOnlinePredictor:
    """
    Class for predicting remaining completion time of task sequences
    """
    def __init__(self, total_tasks: int):
        self.total_tasks = total_tasks
        self.i = 0
        self.sum_j = 0.0
        self.sum_t = 0.0
        self.sum_jt = 0.0
        self.sum_j_square = 0.0
        self.last_timestamp = 0.0

    def add_event(self,
                  timestamp: float):
        """
        Adds the next event with specified timestamp, i.e. some task is processed and
        we are logging its completion time

        Parameters
        ----------
        timestamp: float
            timestamp of the event
        """
        self.i += 1
        j = self.i
        t = timestamp
        self.sum_j += j
        self.sum_t += t
        self.sum_jt += j * t
        self.sum_j_square += j * j
        self.last_timestamp = t

    def expected_remaining_time(self):
        """
        Calculates expected remaining time

        Returns
        -------
            Expected timestamp of the event numbered "total_tasks"
        """
        if self.i < 2:
            return 0.0

        k_denominator = self.i * self.sum_j_square - self.sum_j * self.sum_j
        if k_denominator == 0:
            return 0.0

        k = (self.i * self.sum_jt - self.sum_j * self.sum_t) / k_denominator
        b = (self.sum_t - k * self.sum_j) / self.i

        y = k * self.total_tasks + b
        remaining_time = y - self.last_timestamp

        return remaining_time


In [23]:
eta_predictor = ETAOnlinePredictor(11)
eta_predictor.add_event(0)
for i in range(10):
    eta_predictor.add_event(i + 1)
    print("Expected remaining time:",
          eta_predictor.expected_remaining_time(),
          f" Ref: {9 - i} ",
          "Error" if abs(9 - i - eta_predictor.expected_remaining_time()) > 1e-3 else "Ok")


Expected remaining time: 9.0  Ref: 9  Ok
Expected remaining time: 8.0  Ref: 8  Ok
Expected remaining time: 7.0  Ref: 7  Ok
Expected remaining time: 6.0  Ref: 6  Ok
Expected remaining time: 5.0  Ref: 5  Ok
Expected remaining time: 4.0  Ref: 4  Ok
Expected remaining time: 3.0  Ref: 3  Ok
Expected remaining time: 2.0  Ref: 2  Ok
Expected remaining time: 1.0  Ref: 1  Ok
Expected remaining time: 0.0  Ref: 0  Ok


In [24]:
import time
import random

eta_predictor = ETAOnlinePredictor(11)
eta_predictor.add_event(time.time())
for i in range(10):
    time.sleep(random.uniform(0.5, 1.5))
    eta_predictor.add_event(time.time())
    print("Expected remaining time: ", f"{eta_predictor.expected_remaining_time():.2f}")

Expected remaining time:  8.84
Expected remaining time:  6.68
Expected remaining time:  6.71
Expected remaining time:  6.38
Expected remaining time:  5.56
Expected remaining time:  4.56
Expected remaining time:  3.47
Expected remaining time:  2.42
Expected remaining time:  1.18
Expected remaining time:  0.29
